# Tutorial for real-world data
This tutorial provides step-by-step instructions for reproducing the real-world results presented in our paper. Before running the code, please ensure that you have downloaded the necessary data from our repository.

# Domain clustering

In [ ]:
import torch
import sys
import os
from sklearn.metrics import adjusted_rand_score
# Get the current working directory
current_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.insert(0, os.path.dirname(os.path.dirname(current_dir)))
from SpaMV.spamv import SpaMV
from SpaMV.utils import preprocess_dc, mclust
import scanpy as sc
import matplotlib.pyplot as plt

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
dataname  ='6_Mouse_Embryo'
data_rna = sc.read_h5ad('Data/' + dataname + '/adata_RNA.h5ad')
data_peaks = sc.read_h5ad('Data/' + dataname + '/adata_peaks.h5ad')
omics_names = ['Transcriptomics', 'Epigenomics']
datasets = preprocess_dc([data_rna, data_peaks], omics_names)
n_cluster = 14

# wandb.init(project=dataname)
# wandb.login()
model = SpaMV(datasets, interpretable=False, omics_names=omics_names, device=device)
model.train(dataname)
# wandb.finish()

datasets[0].obsm['SpaMV'] = model.get_embedding()
mclust(datasets[0], n_clusters=n_cluster, key='SpaMV')
fig, axes = plt.subplots(1, 3, figsize=(13.5, 4))  # 1 row, 2 columns

sc.pp.neighbors(datasets[0], use_rep='SpaMV')
sc.tl.umap(datasets[0])
sc.pl.spatial(datasets[0], color='cluster', show=False, title='Annotation', ax=axes[0])
sc.pl.umap(datasets[0], color='SpaMV', ax=axes[1], show=False, s=20, title='UMAP')
sc.pl.spatial(datasets[0], color='SpaMV', show=False, title='SpaMV\nARI: {:.3f}'.format(adjusted_rand_score(datasets[0].obs['cluster'], datasets[0].obs['SpaMV'])), ax=axes[2])
plt.tight_layout()
plt.show()

# Interpretable dimension reduction

In [ ]:
import torch
import sys
import os

from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.sparse import issparse

# Get the current working directory
current_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.insert(0, os.path.dirname(os.path.dirname(current_dir)))
from SpaMV.spamv import SpaMV
from SpaMV.utils import plot_embedding_results, clr_normalize_each_cell, plot_topic_correlation_ratio_multimodal, plot_top_positive_correlations_boxplot
import scanpy as sc
import wandb
import squidpy as sq

def softmax(row):
    shifted = row - np.max(row)  # Prevent overflow
    exp_values = np.exp(shifted)
    return exp_values / exp_values.sum()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
for dataset in ['6_Mouse_Embryo']:
    data_folder = 'Data/' + dataset
    if dataset in ['4_Human_Lymph_Node', '9_Mouse_Thymus', '10_Mouse_Breast_Tumor']:
        data = [sc.read_h5ad(data_folder + '/adata_RNA.h5ad'), sc.read_h5ad(data_folder + '/adata_ADT.h5ad')]
        omics_names = ['Transcriptomics', 'Proteomics']
        zs_dim = 10
        zp_dims = [10, 10]
    elif dataset in ['6_Mouse_Embryo']:
        data = [sc.read_h5ad(data_folder + '/adata_RNA.h5ad'), sc.read_h5ad(data_folder + '/adata_ATAC.h5ad')]
        omics_names = ['Transcriptomics', 'Epigenomics']
        zs_dim = 10
        zp_dims = [20, 5]
    elif dataset in ['7_ME13_1']:
        data = [sc.read_h5ad(data_folder + '/adata_H3K27ac_ATAC.h5ad'), sc.read_h5ad(data_folder + '/adata_H3K27me3_ATAC.h5ad')]
        omics_names = ['H3K27ac', 'H3K27me3']
        zs_dim = 20
        zp_dims = [5, 5]
    elif dataset in ['11_ccRCC_Y7_T']:
        data = [sc.read_h5ad(data_folder + '/adata_RNA.h5ad'), sc.read_h5ad(data_folder + '/adata_MET.h5ad')]
        omics_names = ['Transcriptomics', 'Metabolomics']
        zs_dim = 10
        zp_dims = [5, 5]
    else:
        raise Exception(f"Unkonwn Dataset: {dataset}")
    obs_names = None
    weights = []
    betas = []
    for i in range(len(data)):
        data[i].var_names_make_unique()
        data[i].X = data[i].X.astype(np.float32).toarray() if issparse(data[i].X) else data[i].X.astype(np.float32)
        sc.pp.filter_cells(data[i], min_genes=math.ceil(data[i].n_vars / 100))
        obs_names = data[i].obs_names if obs_names is None else obs_names.intersection(data[i].obs_names)
        weights.append(10 if omics_names[i] == 'Proteomics' else 1)
        betas.append(1)
    data_ori = []
    for i in range(len(data)):
        data[i] = data[i][obs_names]
        data_ori.append(data[i].copy())
        sc.pp.filter_genes(data[i], min_cells=math.ceil(data[i].n_obs / 100))
        if omics_names[i] == 'Transcriptomics':
            data[i] = data[i][:, (data[i].X > 1).sum(0) > data[i].n_obs / 100]
            sc.pp.highly_variable_genes(data[i], subset=False, n_top_genes=1000, flavor='seurat_v3')
            sc.pp.normalize_total(data[i])
            sc.pp.log1p(data[i])
            data[i] = data[i][:, data[i].var_names[data[i].var.highly_variable]]
        elif omics_names[i] == 'Proteomics':
            data[i] = clr_normalize_each_cell(data[i])
        elif omics_names[i] == 'Epigenomics':
            sc.pp.highly_variable_genes(data[i], n_top_genes=1000, subset=True, flavor='seurat')
        elif omics_names[i] == 'Metabolomics':
            sc.pp.normalize_total(data[i])
            sc.pp.log1p(data[i])
            sc.pp.highly_variable_genes(data[i], n_top_genes=1000, subset=True, flavor='seurat')
        else:
            sc.pp.log1p(data[i])
            sq.gr.spatial_neighbors(data[i])
            sq.gr.spatial_autocorr(data[i], mode="moran", genes=data[i].var_names, n_perms=100, n_jobs=1)
            data[i] = data[i][:, data[i].uns['moranI'].index[data[i].uns['moranI']['pval_norm'] < .05]]
            sc.pp.highly_variable_genes(data[i], n_top_genes=1000, subset=True, flavor='seurat')
        data_ori[i] = data_ori[i][:, data[i].var_names]
    # wandb.init(project=dataset)
    # wandb.login()
    model = SpaMV(data, zs_dim=zs_dim, zp_dims=zp_dims, interpretable=True, weights=weights, betas=betas, omics_names=omics_names, device=device, random_seed=0)
    model.train(dataset)
    # wandb.finish()
    z, w = model.get_embedding_and_feature_by_topic()
    z = z.apply(softmax, axis=1)
    plot_embedding_results(data, omics_names, z, w, show=True, save=False, size=100)

    for i in range(len(data)):
        plot_top_positive_correlations_boxplot(data[i], z, omics_names[i])

    plot_topic_correlation_ratio_multimodal(data=data, omics_names=omics_names, z=z)
    plt.show()


# Domain clustering for spatial H3K27ac-H3K27me3 mouse embryo

In [ ]:
import torch
import sys
import os
from sklearn.metrics import adjusted_rand_score
# Get the current working directory
current_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.insert(0, os.path.dirname(os.path.dirname(current_dir)))
from SpaMV.spamv import SpaMV
from SpaMV.utils import preprocess_dc, mclust
import scanpy as sc
import matplotlib.pyplot as plt

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
dataname  ='7_ME13_1'
data_ac = sc.read_h5ad('Data/' + dataname + '/adata_H3K27ac_ATAC.h5ad')
data_me3 = sc.read_h5ad('Data/' + dataname + '/adata_H3K27me3_ATAC.h5ad')
omics_names = ['H3K27ac', 'H3K27me3']
datasets = preprocess_dc([data_ac, data_me3], omics_names)
n_cluster = 21

# wandb.init(project=dataname)
# wandb.login()
model = SpaMV(datasets, interpretable=False, omics_names=omics_names, device=device)
model.train(dataname)
# wandb.finish()

datasets[0].obsm['SpaMV_old'] = model.get_embedding()
mclust(datasets[0], n_clusters=n_cluster, key='SpaMV_old')
fig, axes = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

sc.pp.neighbors(datasets[0], use_rep='SpaMV_old')
sc.tl.umap(datasets[0])
sc.pl.umap(datasets[0], color='SpaMV_old', ax=axes[0], show=False, s=20, title='UMAP')
sc.pl.spatial(datasets[0], color='SpaMV_old', show=False, ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
import torch
import sys
import os

from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.sparse import issparse

# Get the current working directory
current_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.insert(0, os.path.dirname(os.path.dirname(current_dir)))
from SpaMV_old.spamv import SpaMV
from SpaMV_old.utils import plot_embedding_results, clr_normalize_each_cell, plot_topic_correlation_ratio_multimodal, plot_top_positive_correlations_boxplot
import scanpy as sc
import wandb
import squidpy as sq

def softmax(row):
    shifted = row - np.max(row)  # Prevent overflow
    exp_values = np.exp(shifted)
    return exp_values / exp_values.sum()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
for dataset in ['7_ME13_1']:
    data_folder = 'Data/' + dataset
    if dataset in ['4_Human_Lymph_Node', '9_Mouse_Thymus', '10_Mouse_Breast_Tumor']:
        data = [sc.read_h5ad(data_folder + '/adata_RNA.h5ad'), sc.read_h5ad(data_folder + '/adata_ADT.h5ad')]
        omics_names = ['Transcriptomics', 'Proteomics']
        zs_dim = 10
        zp_dims = [10, 10]
    elif dataset in ['6_Mouse_Embryo']:
        data = [sc.read_h5ad(data_folder + '/adata_RNA.h5ad'), sc.read_h5ad(data_folder + '/adata_ATAC.h5ad')]
        omics_names = ['Transcriptomics', 'Epigenomics']
        zs_dim = 10
        zp_dims = [20, 5]
    elif dataset in ['7_ME13_1']:
        data = [sc.read_h5ad(data_folder + '/adata_H3K27ac_ATAC.h5ad'), sc.read_h5ad(data_folder + '/adata_H3K27me3_ATAC.h5ad')]
        omics_names = ['H3K27ac', 'H3K27me3']
        zs_dim = 20
        zp_dims = [5, 5]
    elif dataset in ['11_ccRCC_Y7_T']:
        data = [sc.read_h5ad(data_folder + '/adata_RNA.h5ad'), sc.read_h5ad(data_folder + '/adata_MET.h5ad')]
        omics_names = ['Transcriptomics', 'Metabolomics']
        zs_dim = 10
        zp_dims = [5, 5]
    else:
        raise Exception(f"Unkonwn Dataset: {dataset}")
    obs_names = None
    weights = []
    betas = []
    for i in range(len(data)):
        data[i].var_names_make_unique()
        data[i].X = data[i].X.astype(np.float32).toarray() if issparse(data[i].X) else data[i].X.astype(np.float32)
        sc.pp.filter_cells(data[i], min_genes=math.ceil(data[i].n_vars / 100))
        obs_names = data[i].obs_names if obs_names is None else obs_names.intersection(data[i].obs_names)
        weights.append(10 if omics_names[i] == 'Proteomics' else 1)
        betas.append(1)
    data_ori = []
    for i in range(len(data)):
        data[i] = data[i][obs_names]
        data_ori.append(data[i].copy())
        sc.pp.filter_genes(data[i], min_cells=math.ceil(data[i].n_obs / 100))
        if omics_names[i] == 'Transcriptomics':
            data[i] = data[i][:, (data[i].X > 1).sum(0) > data[i].n_obs / 100]
            sc.pp.highly_variable_genes(data[i], subset=False, n_top_genes=1000, flavor='seurat_v3')
            sc.pp.normalize_total(data[i])
            sc.pp.log1p(data[i])
            data[i] = data[i][:, data[i].var_names[data[i].var.highly_variable]]
        elif omics_names[i] == 'Proteomics':
            data[i] = clr_normalize_each_cell(data[i])
        elif omics_names[i] == 'Epigenomics':
            sc.pp.highly_variable_genes(data[i], n_top_genes=1000, subset=True, flavor='seurat')
        elif omics_names[i] == 'Metabolomics':
            sc.pp.normalize_total(data[i])
            sc.pp.log1p(data[i])
            sc.pp.highly_variable_genes(data[i], n_top_genes=1000, subset=True, flavor='seurat')
        else:
            sc.pp.log1p(data[i])
            sq.gr.spatial_neighbors(data[i])
            sq.gr.spatial_autocorr(data[i], mode="moran", genes=data[i].var_names, n_perms=100, n_jobs=1)
            data[i] = data[i][:, data[i].uns['moranI'].index[data[i].uns['moranI']['pval_norm'] < .05]]
            sc.pp.highly_variable_genes(data[i], n_top_genes=1000, subset=True, flavor='seurat')
        data_ori[i] = data_ori[i][:, data[i].var_names]
    # wandb.init(project=dataset)
    # wandb.login()
    model = SpaMV(data, zs_dim=zs_dim, zp_dims=zp_dims, interpretable=True, weights=weights, betas=betas, omics_names=omics_names, device=device, random_seed=0)
    model.train(dataset)
    # wandb.finish()
    z, w = model.get_embedding_and_feature_by_topic()
    z = z.apply(softmax, axis=1)
    plot_embedding_results(data, omics_names, z, w, show=True, save=False, size=100)

    for i in range(len(data)):
        plot_top_positive_correlations_boxplot(data[i], z, omics_names[i])

    plot_topic_correlation_ratio_multimodal(data=data, omics_names=omics_names, z=z)
    plt.show()
